In [37]:
text_input = ''

In [38]:
  !pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0\
  InstructorEmbedding

In [39]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
import timeit
import sys

In [40]:
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','1acea4e0-c639-4577-b431-6d607796e5d5')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [41]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [42]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="railway-chatbot"

In [43]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [44]:
query = "give me list of trains at 21:40:00 from kudal"
docs=docsearch.similarity_search(query, k=3)
print(docs)

[Document(page_content='13, Station Code: KHED, Station Name: KHED, Arrival Time: 01:52:00,Departure Time: 01:54:00,Distance: 352,Source Station: MAO,Source Station Name: MADGOAN JN.,Destination Station: KOP,Destination Station Name: CHHATRAPATI SHAHU MAHARAJ TERMINUSTrain No: 128, Train Name: MAO-KOP SPEC, SEQ: 14, Station Code: MNI, Station Name: MANGAON, Arrival Time: 04:00:00,Departure Time: 04:02:00,Distance: 422,Source Station: MAO,Source Station Name: MADGOAN JN.,Destination Station: KOP,Destination Station', metadata={}), Document(page_content='13, Station Code: KHED, Station Name: KHED, Arrival Time: 01:52:00,Departure Time: 01:54:00,Distance: 352,Source Station: MAO,Source Station Name: MADGOAN JN.,Destination Station: KOP,Destination Station Name: CHHATRAPATI SHAHU MAHARAJ TERMINUSTrain No: 128, Train Name: MAO-KOP SPEC, SEQ: 14, Station Code: MNI, Station Name: MANGAON, Arrival Time: 04:00:00,Departure Time: 04:02:00,Distance: 422,Source Station: MAO,Source Station Name: MA

In [45]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that I don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [63]:
chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':1024,
                          'temperature':0.8})

In [64]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 2}),return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [65]:
text = "kudal"
lang="en"

In [66]:
import googletrans
from googletrans import Translator

trans = Translator()

if lang=="kn-IN":
    lang="kn"

elif lang=="hi-IN":
    lang="hi"

if lang=="kn":
    target_lang = "en"
    text = trans.translate(text, src=lang, dest=target_lang).text

elif lang=="hi":
    target_lang = "en"
    text = trans.translate(text, src=lang, dest=target_lang).text

print(text)

kudal


In [67]:
user_input = text
if user_input=='exit':
    print('Exiting')
    sys.exit()
result=qa({"query": user_input})
print("Response : ", result["result"])

Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o

KeyboardInterrupt: 